In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from scripts.transformers_JuanPablo import CheckColumnNames,UnknownToZero,FixRanges
from scripts.transformer_Alfredo import FillNaNsWithCeros
from scripts.transformers_Demian import OneHotCodificador
from scripts.transformer_Gonzalo import VectorizarTexto
from scripts.transformer_camila import DefineLimits

In [3]:
data = pd.read_csv("data/datos_grasas_Tec_limpios.csv")
data_pdfs = pd.read_csv("data/datos_grasas_pdfs_limpios.csv")

In [4]:
# Categorical columns to be one-hot encoded
categorical_columns = [
    "Aceite Base",
    "Espesante",
    "Clasificacion ISO 6743-9",
    "color",
    "textura"
]
range_columns = ["Punto de Gota, °C", "Estabilidad Mecánica, %", "Carga Timken Ok, lb", "Resistencia al Lavado por Agua a 80°C, %"]

In [5]:
process = Pipeline(steps=[
    ("To have columns names needed", CheckColumnNames()),
    ("To change unkown data to zeros", UnknownToZero("Grado NLGI Consistencia")),
    ("To fix ranges and single values", FixRanges("Penetración de Cono a 25°C, 0.1mm")),
    ("To define limits", DefineLimits(columns=range_columns, margin=10, verbose=False)),
    ("OneHot_categoricals", OneHotCodificador(columns=categorical_columns,drop_original=True,dtype=int)),
    ("Vectorizar subtitulo", VectorizarTexto("subtitulo")),
    ("Vectorizar descripcion", VectorizarTexto("descripcion")),
    ("Vectorizar beneficios", VectorizarTexto("beneficios")),
    ("Vectorizar aplicaciones", VectorizarTexto("aplicaciones")),
    ("To fill NaNs with zeros", FillNaNsWithCeros()),
    ('MinMax', ColumnTransformer(transformers=[('MinMax', MinMaxScaler(), slice(1,None))]))

])

In [6]:
data.head()

,idDatosGrasas,Aceite Base,Espesante,Grado NLGI Consistencia,Viscosidad del Aceite Base a 40°C. cSt,"Penetración de Cono a 25°C, 0.1mm","Punto de Gota, °C","Estabilidad Mecánica, %","Punto de Soldadura Cuatro Bolas, kgf","Desgaste Cuatro Bolas, mm",...,Clasificacion ISO 6743-9,Factor de Velocidad,"Temperatura de Servicio °C, min","Temperatura de Servicio °C, max",subtitulo,descripcion,beneficios,aplicaciones,color,textura
0,1,Semi-Sintetico,Complejo Sulfonato de Calcio,2.0,680.0,279,304,1.07,500.0,NaN,...,NaN,200000.0,5,220,Grasa de servicio pesado para alta resistencia...,El producto es grasa lubricante de servicio pe...,Excelentes caracterÃ­sticas de resistencia al ...,Debido a su composiciÃ³n Ã³ptima de aceites de...,CafÃ©,Suave
1,2,Mineral HT,Complejo Sulfonato de Calcio,1.5,460.0,300,304,2.6,900.0,0.50,...,NaN,300000.0,-30,150,Grasa para lubricaciÃ³n de equipo pesado.,El producto es una grasa lubricante de gran ad...,Extremo soporte de carga.\n@Alto contenido de ...,LubricaciÃ³n centralizada de equipo pesado.\n@...,Negro,Adherente
2,3,Mineral HT,Complejo Sulfonato de Calcio,2.0,460.0,265 - 295,300,10,500.0,0.48,...,NaN,NaN,-10,150,Grasa para lubricaciÃ³n de equipo pesado.,El producto es una grasa lubricante de gran ad...,Extremo soporte de carga.\n@Alto contenido de ...,LubricaciÃ³n centralizada de equipo pesado.\n@...,Negro,Adherente
3,4,Mineral HT,Complejo Sulfonato de Calcio,2.0,220.0,266 - 295,300,5,500.0,0.45,...,NaN,NaN,-15,150,Grasa lubricante para condiciones extremas.,El producto es una grasa lubricante elaborada ...,Alta estabilidad tÃ©rmica y anti-oxidante.\n@E...,Rodamientos expuestos a alta temperatura y car...,Ambar,Suave
4,5,Mineral HT,Complejo de Aluminio - Poliurea,1.5,680.0,295,260,6.5,620.0,0.48,...,NaN,NaN,-5,150,Grasa lubricante para servicio pesado con alta...,El producto es un grasa lubricante con propied...,Excelente soporte de carga extrema.\n@Excelent...,El producto es ideal para la lubricaciÃ³n de m...,Negro,Adherente


In [7]:
X=process.fit_transform(data)
X

array([[0.66666667, 0.17894737, 0.55555556, ..., 0.        , 0.        ,
        1.        ],
       [0.5       , 0.12105263, 1.        , ..., 0.926057  , 0.        ,
        0.        ],
       [0.66666667, 0.12105263, 0.55555556, ..., 0.926057  , 0.        ,
        0.        ],
       ...,
       [1.        , 0.01052632, 0.27777778, ..., 0.        , 0.        ,
        0.        ],
       [0.33333333, 0.05789474, 0.35      , ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.05789474, 0.35      , ..., 0.        , 0.        ,
        0.        ]], shape=(51, 1418))

In [8]:
Y=process.transform(data_pdfs)
Y[0]

array([0.66666667, 0.025     , 0.45311111, ..., 0.        , 0.        ,
       0.        ], shape=(1418,))

In [9]:
data.columns

Index(['idDatosGrasas', 'Aceite Base', 'Espesante', 'Grado NLGI Consistencia',
       'Viscosidad del Aceite Base a 40°C. cSt', 'Punto de Gota, °C',
       'Estabilidad Mecánica, %', 'Punto de Soldadura Cuatro Bolas, kgf',
       'Desgaste Cuatro Bolas, mm', 'Carga Timken Ok, lb',
       'Resistencia al Lavado por Agua a 80°C, %', 'Clasificacion ISO 6743-9',
       'Factor de Velocidad', 'Temperatura de Servicio °C, min',
       'Temperatura de Servicio °C, max', 'subtitulo', 'descripcion',
       'beneficios', 'aplicaciones', 'color', 'textura',
       'Penetración de Cono a 25°C, 0.1mm min',
       'Penetración de Cono a 25°C, 0.1mm max'],
      dtype='object')

In [10]:
data_pdfs.columns

Index(['idDatosGrasas', 'Aceite Base', 'Espesante', 'Grado NLGI Consistencia',
       'Viscosidad del Aceite Base a 40°C. cSt', 'Punto de Gota, °C',
       'Estabilidad Mecánica, %', 'Punto de Soldadura Cuatro Bolas, kgf',
       'Desgaste Cuatro Bolas, mm', 'Carga Timken Ok, lb',
       'Resistencia al Lavado por Agua a 80°C, %', 'Clasificacion ISO 6743-9',
       'Factor de Velocidad', 'Temperatura de Servicio °C, min',
       'Temperatura de Servicio °C, max', 'subtitulo', 'descripcion',
       'beneficios', 'aplicaciones', 'color', 'textura',
       'Penetración de Cono a 25°C, 0.1mm min',
       'Penetración de Cono a 25°C, 0.1mm max'],
      dtype='object')

In [11]:
set(data_pdfs.columns).difference(data.columns)

set()